# Data Cleaning - Last Dance

Notebook này tập trung vào bước **làm sạch dữ liệu thô** trước khi preprocessing:

- **Tất cả file input và output đều nằm trong folder `data/`**:
  - Input: `data/admission.csv`, `data/academic_records.csv`
  - Output: `data/cleaned_admission.csv`, `data/final_combined_table(2) (1).csv`, `data/final_combined_cleaned.csv`
- Các file output sẽ được dùng trong notebook `data_preprocessing.ipynb`.

In [2]:
import os
import random
import numpy as np
import pandas as pd

# Seed cố định để kết quả reproducible
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)

## I. Tìm hiểu về dữ liệu

In [3]:
# I.1 Load admission.csv

admission_df = pd.read_csv(os.path.join(DATA_DIR, 'admission.csv'))

print('admission shape:', admission_df.shape)
print('\nCác cột:')
print(admission_df.head())

admission shape: (30217, 6)

Các cột:
       MA_SO_SV  NAM_TUYENSINH PTXT TOHOP_XT  DIEM_TRUNGTUYEN  DIEM_CHUAN
0  0570116c3448           2018    5      A00            15.86       15.10
1  921cfe1e9ca9           2018    5      A01            15.98       14.98
2  8aeb1516c333           2018    5      A01            17.05       15.18
3  94ff3745a70e           2018    5      A00            18.67       14.78
4  5c5900eb2b53           2018    5      D07            16.67       15.06


In [4]:
# I.2 Thông tin về bộ dữ liệu admission

print('Info:')
admission_df.info()

print('\nDescribe:')
print(admission_df.describe())

print('\nCount:')
print(admission_df.count())

total_student_distinct = admission_df['MA_SO_SV'].nunique()
print(f'\nTotal number of students: {total_student_distinct}')

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30217 entries, 0 to 30216
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MA_SO_SV         30217 non-null  object 
 1   NAM_TUYENSINH    30217 non-null  int64  
 2   PTXT             30217 non-null  object 
 3   TOHOP_XT         30217 non-null  object 
 4   DIEM_TRUNGTUYEN  30217 non-null  float64
 5   DIEM_CHUAN       30217 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.4+ MB

Describe:
       NAM_TUYENSINH  DIEM_TRUNGTUYEN    DIEM_CHUAN
count   30217.000000      30217.00000  30217.000000
mean     2021.790979         23.09548     21.173715
std         2.309299          3.76811      3.475354
min      2018.000000          0.00000      0.000000
25%      2020.000000         21.13000     18.940000
50%      2022.000000         23.30000     21.610000
75%      2024.000000         24.92000     23.580000
max      2025.000000         

## II. Xử lý admission.csv

**Lưu ý:** Giữ toàn bộ dữ liệu admission, không loại bỏ record nào. Merge toàn bộ vào academic_records.

In [5]:
# II.1 Giữ toàn bộ admission (không loại bỏ record nào)

# Giữ toàn bộ dữ liệu admission, không filter
admission_cleaned = admission_df.copy()
print(f'Total admission records: {len(admission_cleaned)}')
print('Note: Không loại bỏ record nào, merge toàn bộ vào academic_records')

Total admission records: 30217
Note: Không loại bỏ record nào, merge toàn bộ vào academic_records


In [6]:
# II.2 Thống kê top 15 tổ hợp xét tuyển ít được lựa chọn nhất

top_15_least_common = admission_cleaned['TOHOP_XT'].value_counts().nsmallest(15)
print('Top 15 tổ hợp xét tuyển ít được lựa chọn nhất và số lượng sinh viên ứng với mỗi tổ hợp:')
print(top_15_least_common)

Top 15 tổ hợp xét tuyển ít được lựa chọn nhất và số lượng sinh viên ứng với mỗi tổ hợp:
TOHOP_XT
SP1      1
CCQT     1
VS4      1
TT       2
D24      2
VS1      2
K02      2
H07      4
D29      6
K01      6
C02     11
V10     17
A02     23
X03     24
H01     37
Name: count, dtype: int64


In [7]:
# II.3 Lưu admission đã clean

out_path_admission = os.path.join(DATA_DIR, 'cleaned_admission.csv')
admission_cleaned.to_csv(out_path_admission, index=False)
print(f'Cleaned data saved to {out_path_admission}')

Cleaned data saved to data/cleaned_admission.csv


## III. Tạo final_combined_table từ academic_records + cleaned_admission

In [10]:
# III.1 Load academic_records.csv

academic_df = pd.read_csv(os.path.join(DATA_DIR, 'academic_records.csv'))

print('academic_records shape:', academic_df.shape)
print('\nInfo:')
academic_df.info()

unique_students = academic_df['MA_SO_SV'].nunique()
print(f'\nUnique students in academic_records: {unique_students}')

academic_records shape: (105726, 6)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105726 entries, 0 to 105725
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   MA_SO_SV      105726 non-null  object 
 1   HOC_KY        105726 non-null  object 
 2   CPA           105726 non-null  float64
 3   GPA           105726 non-null  float64
 4   TC_DANGKY     105726 non-null  int64  
 5   TC_HOANTHANH  105726 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 4.8+ MB

Unique students in academic_records: 20381


In [13]:
# III.2 Biến đổi HOC_KY thành Hoc_Ky và Nam_Hoc

df_academic = academic_df.copy()

# Tách HOC_KY thành Hoc_Ky_Label và Nam_Hoc
df_academic[['Hoc_Ky_Label', 'Nam_Hoc']] = df_academic['HOC_KY'].str.split(' ', n=1, expand=True)

# Chuyển HK1, HK2 thành 1, 2
df_academic['Hoc_Ky'] = df_academic['Hoc_Ky_Label'].map({'HK1': 1, 'HK2': 2}).astype(int)
df_academic = df_academic.drop(columns=['Hoc_Ky_Label'])


In [14]:


print("=" * 80)
print("GIẢI THÍCH VỀ CẤU TRÚC DỮ LIỆU")
print("=" * 80)
print("\nDữ liệu academic_records.csv chứa LỊCH SỬ HỌC TẬP của sinh viên qua các năm.")
print("Mỗi sinh viên có NHIỀU RECORDS tương ứng với các học kỳ khác nhau.\n")

example_student = df_academic['MA_SO_SV'].iloc[0]
student_records = df_academic[df_academic['MA_SO_SV'] == example_student].sort_values(['Nam_Hoc', 'Hoc_Ky'])
print(f"Ví dụ: Lịch sử học tập của sinh viên {example_student}:")
print(student_records[['MA_SO_SV', 'Hoc_Ky', 'Nam_Hoc', 'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH']])
print(f"\n→ Sinh viên này có {len(student_records)} records (tương ứng với {len(student_records)} học kỳ)")


print("\n" + "=" * 80)
print("THỐNG KÊ:")
print("=" * 80)
print(f"Tổng số records: {len(df_academic)}")
print(f"Tổng số sinh viên: {df_academic['MA_SO_SV'].nunique()}")
print(f"Số records trung bình mỗi sinh viên: {len(df_academic) / df_academic['MA_SO_SV'].nunique():.2f}")

print("\nPhân bố theo năm học:")
print(df_academic['Nam_Hoc'].value_counts().sort_index())

print("\nPhân bố theo học kỳ:")
print(df_academic['Hoc_Ky'].value_counts().sort_index())

print("\n" + "=" * 80)

GIẢI THÍCH VỀ CẤU TRÚC DỮ LIỆU

Dữ liệu academic_records.csv chứa LỊCH SỬ HỌC TẬP của sinh viên qua các năm.
Mỗi sinh viên có NHIỀU RECORDS tương ứng với các học kỳ khác nhau.

Ví dụ: Lịch sử học tập của sinh viên f022ed8d1ac1:
           MA_SO_SV  Hoc_Ky    Nam_Hoc   CPA   GPA  TC_DANGKY  TC_HOANTHANH
39436  f022ed8d1ac1       1  2020-2021  0.99  1.66         15             9
0      f022ed8d1ac1       2  2020-2021  2.19  2.02         18            18
79003  f022ed8d1ac1       1  2021-2022  2.37  2.19         20            20
4658   f022ed8d1ac1       2  2021-2022  1.89  2.09         16            16
1      f022ed8d1ac1       1  2022-2023  0.95  2.12         14             7
3      f022ed8d1ac1       2  2022-2023  1.37  1.93         26            23
2      f022ed8d1ac1       1  2023-2024  0.81  1.89         29            16
4      f022ed8d1ac1       2  2023-2024  1.71  1.91         16            13

→ Sinh viên này có 8 records (tương ứng với 8 học kỳ)

THỐNG KÊ:
Tổng số records: 10572

In [15]:
# III.3 Tách thông tin năm học và tính tỷ lệ hoàn thành

df_academic['Nam_Bat_Dau_Hoc'] = df_academic['Nam_Hoc'].str.split('-').str[0].astype(int)
df_academic['Nam_Ket_Thuc_Hoc'] = df_academic['Nam_Hoc'].str.split('-').str[1].astype(int)

# Tỷ lệ hoàn thành tín chỉ
df_academic['Ti_Le_Hoan_Thanh_Tin_Chi'] = ((df_academic['TC_HOANTHANH'] / df_academic['TC_DANGKY']) * 100).astype(str) + '%'

print('Sau khi thêm Nam_Bat_Dau_Hoc, Nam_Ket_Thuc_Hoc, Ti_Le_Hoan_Thanh_Tin_Chi:')
print(df_academic[['MA_SO_SV', 'Hoc_Ky', 'Nam_Hoc', 'TC_DANGKY', 'TC_HOANTHANH', 'Ti_Le_Hoan_Thanh_Tin_Chi']].head())

Sau khi thêm Nam_Bat_Dau_Hoc, Nam_Ket_Thuc_Hoc, Ti_Le_Hoan_Thanh_Tin_Chi:
       MA_SO_SV  Hoc_Ky    Nam_Hoc  TC_DANGKY  TC_HOANTHANH  \
0  f022ed8d1ac1       2  2020-2021         18            18   
1  f022ed8d1ac1       1  2022-2023         14             7   
2  f022ed8d1ac1       1  2023-2024         29            16   
3  f022ed8d1ac1       2  2022-2023         26            23   
4  f022ed8d1ac1       2  2023-2024         16            13   

  Ti_Le_Hoan_Thanh_Tin_Chi  
0                   100.0%  
1                    50.0%  
2      55.172413793103445%  
3       88.46153846153845%  
4                   81.25%  


In [17]:
# III.4 Merge với admission đã clean để tạo final_combined_cleaned.csv

combined_table = df_academic.merge(admission_cleaned, on='MA_SO_SV', how='left')

# Sắp xếp lại thứ tự cột
cols_order = ['MA_SO_SV', 'Hoc_Ky', 'Nam_Hoc', 'Nam_Bat_Dau_Hoc', 'Nam_Ket_Thuc_Hoc', 
              'CPA', 'GPA', 'TC_DANGKY', 'TC_HOANTHANH', 'Ti_Le_Hoan_Thanh_Tin_Chi',
              'NAM_TUYENSINH', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
cols_order = [c for c in cols_order if c in combined_table.columns]
combined_table = combined_table[cols_order]

out_path_table = os.path.join(DATA_DIR, 'final_combined_cleaned.csv')
combined_table.to_csv(out_path_table, index=False)
print(f'Đã lưu {out_path_table}: {combined_table.shape}')

Đã lưu data/final_combined_cleaned.csv: (105726, 15)
